# Introduction

This notebook demonstrates how to carry out an ordering of a disordered structure using pymatgen.

In [101]:
import warnings
warnings.filterwarnings('ignore')

In [103]:
# Let us start by creating a disordered CuAu fcc structure.
from pymatgen.core import Lattice, Structure

# specie = {"Cu0+": 0.5, "Au0+": 0.5}
# cuau = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.677), [specie], [[0, 0, 0]])
cuau = Structure.from_file('/content/LMTOF_order_Disorder_transform_to_primitive.cif')
cuau.add_oxidation_state_by_element({"Li": +1, "Mn": +3,"Ti":+4,"O":-2,"F":-1})
print(cuau)
cuau

Full Formula (Li0.6 Ti0.2 Mn0.2 O0.6 F0.4)
Reduced Formula: Li0.6Ti0.2Mn0.2O0.6F0.4
abc   :   2.974374   2.974374   2.974374
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (2)
  #  SP                             a    b    c
---  ---------------------------  ---  ---  ---
  0  Li+:0.6, Ti4+:0.2, Mn3+:0.2  0    0    0
  1  O2-:0.6, F-:0.4              0.5  0.5  0.5


<IPython.core.display.Javascript object>

Note that each site is now 50% occupied by Cu and Au. Because the ordering algorithms uses an Ewald summation to rank the structures, you need to explicitly specify the oxidation state for each species, even if it is 0. Let us now perform ordering of these sites using two methods.

## Method 1 - Using the OrderDisorderedStructureTransformation

The first method is to use the OrderDisorderedStructureTransformation.

In [63]:
from pymatgen.transformations.standard_transformations import SupercellTransformation


supercell_transformation = SupercellTransformation.from_scaling_factors(1, 1, 5)
supercell_structure = supercell_transformation.apply_transformation(cuau)
cuau = supercell_structure

In [89]:
from pymatgen.transformations.standard_transformations import (
    OrderDisorderedStructureTransformation,
)

trans = OrderDisorderedStructureTransformation()

ss = trans.apply_transformation(cuau, return_ranked_list=False)

print(len(ss))

# print(ss[0])

10


Note that the OrderDisorderedTransformation (with a sufficiently large return_ranked_list parameter) returns all orderings, including duplicates without accounting for symmetry. A computed ewald energy is returned together with each structure. To eliminate duplicates, the best way is to use StructureMatcher's group_structures method, as demonstrated below.

In [100]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import Structure, Lattice
structure = ss
sga_default = SpacegroupAnalyzer(structure)
sg_symbol, sg_number, sg_crystal = sga_default.get_space_group_symbol(), sga_default.get_space_group_number(),sga_default.get_crystal_system()


print(f"crystal:{sg_crystal}, symbol: {sg_symbol}, number: {sg_number}")

crystal:trigonal, symbol: R3m, number: 160


In [96]:
ss.to('lowest.cif')

"# generated using pymatgen\ndata_Li3TiMnO3F2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   2.97437400\n_cell_length_b   2.97437400\n_cell_length_c   14.87187000\n_cell_angle_alpha   60.00000000\n_cell_angle_beta   60.00000000\n_cell_angle_gamma   60.00000000\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   Li3TiMnO3F2\n_chemical_formula_sum   'Li3 Ti1 Mn1 O3 F2'\n_cell_volume   93.03400879\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_type_symbol\n _atom_type_oxidation_number\n  Li+  1.0\n  Ti4+  4.0\n  Mn3+  3.0\n  O2-  -2.0\n  F-  -1.0\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Li+  TI  1  0.00000000  0.00000000  0.00000000  1\n  Li+  TI  1  0.00000000  0.00000000  0.20000000  1\n  Li+  TI  1  0.00000000  0.00000000  0.60000000  1\n  Ti4+  TI  1  0.

In [90]:
from pymatgen.analysis.structure_matcher import StructureMatcher

matcher = StructureMatcher()
groups = matcher.group_structures([d["structure"] for d in ss])
print(len(groups))
print(groups[0][0])

KeyError: "Invalid key='structure'"

In [86]:
for i in range(len(groups)):
  print(groups[i][])

Full Formula (Li3 Ti1 Mn1 O3 F2)
Reduced Formula: Li3TiMnO3F2
abc   :   2.974374   2.974374  14.871870
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (10)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Li+   0    0    0
  1  Li+   0    0    0.4
  2  Li+   0    0    0.6
  3  Ti4+  0    0    0.8
  4  Mn3+  0    0    0.2
  5  O2-   0.5  0.5  0.1
  6  O2-   0.5  0.5  0.7
  7  O2-   0.5  0.5  0.9
  8  F-    0.5  0.5  0.3
  9  F-    0.5  0.5  0.5
Full Formula (Li3 Ti1 Mn1 O3 F2)
Reduced Formula: Li3TiMnO3F2
abc   :   2.974374   2.974374  14.871870
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (10)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Li+   0    0    0
  1  Li+   0    0    0.4
  2  Li+   0    0    0.6
  3  Ti4+  0    0    0.8
  4  Mn3+  0    0    0.2
  5  O2-   0.5  0.5  0.3
  6  O2-   0.5  0.5  0.7
  7  O2-   0.5  0.5  0.9
  8  F-    0.5  0.5  0.1
  9  F-    0.5  0.5  0.5
Full Formula

In [82]:
for i in range(len(groups)):
  for j in range(len(groups[i])):
    groups[i][j].to(filename=f"/content/drive/MyDrive/LMTOF_Disorder_to_Order/CIFS/CIF_{i}_{j}.cif")
  # groups[i].to(filename="/content/drive/MyDrive/LMTOF_Disorder_to_Order/CIFS/CIF_%d.cif" % i)

In [83]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import Structure, Lattice
from tabulate import tabulate

# for i in range(len(ss)):
#   structure = Structure.from_file(f"/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_{i}.cif")
#   sga_default = SpacegroupAnalyzer(structure)
#   sg_symbol, sg_number,sg_crystal = sga_default.get_space_group_symbol(), sga_default.get_space_group_number(),sga_default.get_crystal_system()
# # structure = Structure.from_file("/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_%d.cif")
# # sga_default = SpacegroupAnalyzer(structure)
# # sg_symbol, sg_number = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

#   # print(f"crystal: {sg_crystal}  spacegroup: symbol: {sg_symbol}, number: {sg_number}")

#   data.append([sg_crystal,sg_symbol,sg_number])
# print(tabulate(data, headers=["crystal","spacegroup","number"]))
for i in range(len(groups)):
  data = []
  for j in range(len(groups[i])):
    structure = Structure.from_file(f"/content/drive/MyDrive/LMTOF_Disorder_to_Order/CIFS/CIF_{i}_{j}.cif")
    sga_default = SpacegroupAnalyzer(structure)
    print(i,j)
    sg_symbol, sg_number,sg_crystal = sga_default.get_space_group_symbol(), sga_default.get_space_group_number(),sga_default.get_crystal_system()
    data.append([sg_crystal,sg_symbol,sg_number])
  print(tabulate(data, headers=["crystal","spacegroup","number"]))
    # groups[i][j].to(filename=f"/content/drive/MyDrive/LMTOF_Disorder_to_Order/CIFS/CIF_{i}_{j}.cif")
  # groups[i].to(filename="/content/drive/MyDrive/LMTOF_Disorder_to_Order/CIFS/CIF_%d.cif" % i)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
crystal    spacegroup      number
---------  ------------  --------
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
crystal    spacegroup      number
---------  ------------  --------
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
trigonal   R3m                160
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
crystal    spacegroup      number
---------  ------------  -----

## Method 2 - Using the EnumerateStructureTransformation

If you have enumlib installed, you can use the EnumerateStructureTransformation. This automatically takes care of symmetrically equivalent orderings and can enumerate supercells, but is much more prone to parameter sensitivity and cannot handle very large structures. The example below shows an enumerate of CuAu up to cell sizes of 4.

In [43]:
from pymatgen.transformations.advanced_transformations import (
    EnumerateStructureTransformation,
)

# specie = {"Cu": 0.5, "Au": 0.5}
# cuau = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.677), [specie], [[0, 0, 0]])

trans = EnumerateStructureTransformation(max_cell_size=100)
ss = trans.apply_transformation(cuau, return_ranked_list=1000)

In [46]:
print(len(ss))
print("cell sizes are %s" % ([len(d["structure"]) for d in ss]))

90
cell sizes are [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# The following two lines write all the structures to CIF files with names CuAu_0.cif, CuAu_1.cif, ...
for i, d in enumerate(ss):
    d["structure"].to(filename="/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_%d.cif" % i)

In [57]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import Structure, Lattice
from tabulate import tabulate
data = []
for i in range(len(ss)):
  structure = Structure.from_file(f"/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_{i}.cif")
  sga_default = SpacegroupAnalyzer(structure)
  sg_symbol, sg_number,sg_crystal = sga_default.get_space_group_symbol(), sga_default.get_space_group_number(),sga_default.get_crystal_system()
# structure = Structure.from_file("/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_%d.cif")
# sga_default = SpacegroupAnalyzer(structure)
# sg_symbol, sg_number = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

  # print(f"crystal: {sg_crystal}  spacegroup: symbol: {sg_symbol}, number: {sg_number}")

  data.append([sg_crystal,sg_symbol,sg_number])
print(tabulate(data, headers=["crystal","spacegroup","number"]))

crystal       spacegroup      number
------------  ------------  --------
monoclinic    Cm                   8
orthorhombic  Imm2                44
tetragonal    I4mm               107
trigonal      R3m                160
tetragonal    I4mm               107
tetragonal    I4mm               107
monoclinic    Cm                   8
trigonal      R3m                160
orthorhombic  Imm2                44
orthorhombic  Imm2                44
trigonal      R3m                160
tetragonal    I4mm               107
monoclinic    Cm                   8
monoclinic    Cm                   8
monoclinic    Cm                   8
orthorhombic  Imm2                44
tetragonal    I4mm               107
monoclinic    Cm                   8
monoclinic    Cm                   8
tetragonal    I4mm               107
orthorhombic  Imm2                44
monoclinic    Cm                   8
tetragonal    I4mm               107
monoclinic    Cm                   8
trigonal      R3m                160
m

Note that structures with cell sizes ranging from 1-3x the unit cell size is generated.

## Conclusion

This notebook illustrates two basic ordering/enumeration approaches. In general, OrderDisorderedTransformation works better for large cells and is useful if you need just any quick plausible ordering. EnumerateStructureTransformation is more rigorous, but is prone to sensitivity errors and may require fiddling with various parameters.